#<font color="yellow">This sample shows how a forecast can be uploaded to Kyriba and reconciled against a bank statement </font>
In the sample, one should fill a form to create a forecast in Kyriba. Based on the input information, a bank statement file is generated in CAMT053 format and then uploaded into Kyriba. Once the bank statement is integrated, Kyriba reconciles the forecast with the transactions in the bank statement, so that the status of the forecast is changed to "Actual".

## <font color='yellow'>Code for token generation and API management</font>

In [ ]:
#@title Select **servicePack** the Sample should run
from subprocess import getstatusoutput
servicePack = "DEMO" 

if servicePack == 'DEMO':
  branch = 'main'
else:
  branch = servicePack
print(branch)
rm = getstatusoutput("rm Colab_shared_code_for_API_usage -rf ") 
clone = getstatusoutput("git clone -l -s --branch " +  branch + " https://github.com/kyriba/Colab_shared_code_for_API_usage.git") 
import importlib
enableFormatterDF = True #@param {type:"boolean"}
from google.colab import data_table
if enableFormatterDF is True:
  data_table.enable_dataframe_formatter()
else:
  data_table.disable_dataframe_formatter()
try:  
  importlib.reload(sample_requests)
except:
  print()
from Colab_shared_code_for_API_usage import sample_requests

In [ ]:
#@title Import custom modules
import json as js
import base64
import time
from datetime import datetime, date

In [ ]:
#@title Connect and get token { display-mode: "form" }
token = sample_requests.login()


## <font color='yellow'>Import Forecast</font>

In [ ]:
#@title <font color='lime'> Create a forecast</font>
#@markdown Provide values for parameters below:
account = 'ST_ACC01' #@param {type:"string"}
flowCode = '+WIRE' #@param {allow-input: true} {type: "string"}
budgetCode = '+WIRE' #@param {type:"string"} 
status = 'CONFIRMED' #@param ["ESTIMATED", "CONFIRMED"] {type:"string"}
transactionDate = '2022-11-29' #@param {allow-input: true} {type:"date"} 
valueDate = '2022-11-29' #@param {allow-input: true} {type:"date"} 
accountingDate = '2022-11-29' #@param {allow-input: true} {type:"date"} 
currency = 'EUR' #@param {allow-input: true} {type:"string"} 
amount = 17 #@param {allow-input: true} {type:"number"} 
description = 'Inbound flow forecast' #@param {type:"string"} 
reference = 'REF00178' #@param {allow-input: true} {type:"string"} 


checker = 'false'

payload = sample_requests.js.dumps({
  "account": {
    "code": account
  },
  "flowCode": {
    "code": flowCode
  },
  "budgetCode": {
    "code": budgetCode
  },
  "status": status,
  "date": {
    "transactionDate": transactionDate,
    "valueDate": valueDate,
    "accountingDate": accountingDate
  },
  "flowAmount": {
    "currency": {
      "code": currency
    },
    "amount": amount
  },
  "accountAmount": {
    "accountAmount": amount,
  },
  "description": description,
  "reference": reference,
})


#print(payload)

cfg = sample_requests.pd.read_csv("/content/config.csv")
client_id = cfg['client_id'].values[0]
test = client_id.split('@')
client_id = test[0]
client_id

# create forecast in json 
try: 
  res2 = sample_requests.post_results(token, '/v1/cash-flows', payload, {}, {'Content-Type': 'application/json'})
  print('\nForecast uuid: ' + res2['uuid'])
  try:
    if res2['uuid']:
      checker = 'true'
      print('\nForecast with the reference "' + reference + '" has been succesfully created in Kyriba.')
      print()
      # print(res2)
  except:
    print()
    # print('Error')
    print("\nForecast could not be created in Kyriba.")
except Exception2 as e2:
  print("Forecast could not be created in Kyriba.")

## <font color='yellow'>Import Bank statement</font>

In [ ]:
#@title CAMT53 template

camt53 = '''<?xml version="1.0" encoding="UTF-8"?>
<Document xsi:schemaLocation="urn:iso:std:iso:20022:tech:xsd:camt.053.001.04 camt.053.001.04.xsd"
	xmlns="urn:iso:std:iso:20022:tech:xsd:camt.053.001.04"
	xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
	<BkToCstmrStmt>
		<GrpHdr>
			<MsgId>602577997205112</MsgId>
			<CreDtTm>2022-03-18T21:15:02</CreDtTm>
			<MsgRcpt>
				<Id>
					<OrgId>
						<AnyBIC>BCVLFR2LXXX</AnyBIC>
					</OrgId>
				</Id>
			</MsgRcpt>
			<MsgPgntn>
				<PgNb>1</PgNb>
				<LastPgInd>true</LastPgInd>
			</MsgPgntn>
		</GrpHdr>
		<Stmt>
			<Id>402077308205112</Id>
			<ElctrncSeqNb>7</ElctrncSeqNb>
			<CreDtTm>2022-03-18T21:15:02</CreDtTm>
			<FrToDt>
				<FrDtTm>2022-03-18T00:00:00</FrDtTm>
				<ToDtTm>2022-03-18T00:00:00</ToDtTm>
			</FrToDt>
			<Acct>
				<Id>
					<IBAN>FR3409876098551233456781766</IBAN>
				</Id>
				<Ccy>EUR</Ccy>
				<Ownr>
					<Nm>Kyriba</Nm>
				</Ownr>
			</Acct>
			<Bal>
				<Tp>
					<CdOrPrtry>
						<Cd>OPBD</Cd>
					</CdOrPrtry>
				</Tp>
				<Amt Ccy="EUR">2695993.91</Amt>
				<CdtDbtInd>CRDT</CdtDbtInd>
				<Dt>
					<Dt>2022-03-18</Dt>
				</Dt>
			</Bal>
			<Bal>
				<Tp>
					<CdOrPrtry>
						<Cd>CLBD</Cd>
					</CdOrPrtry>
				</Tp>
				<Amt Ccy="EUR">1247358.85</Amt>
				<CdtDbtInd>CRDT</CdtDbtInd>
				<Dt>
					<Dt>2022-03-18</Dt>
				</Dt>
			</Bal>
			<Ntry>
				<Amt Ccy="CHF">100.00</Amt>
				<CdtDbtInd>CRDT</CdtDbtInd>
				<Sts>BOOK</Sts>
				<BookgDt>
					<Dt>2022-10-17</Dt>
				</BookgDt>
				<ValDt>
					<Dt>2022-10-17</Dt>
				</ValDt>
				<AcctSvcrRef>793616889522077/2</AcctSvcrRef>
				<BkTxCd>
					<Domn>
						<Cd>PMNT</Cd>
						<Fmly>
							<Cd>APII</Cd>
							<SubFmlyCd>TEST</SubFmlyCd>
						</Fmly>
					</Domn>
				</BkTxCd>
				<NtryDtls>
					<TxDtls>
						<Refs>
							<MsgId>API CF Rec</MsgId>
							<InstrId>Instruction ID</InstrId>
							<EndToEndId>Reference</EndToEndId>
						</Refs>
						<Amt Ccy="EUR">100.00</Amt>
						<CdtDbtInd>CRDT</CdtDbtInd>
						<RltdPties>
							<Dbtr>
								<Nm>Kyriba</Nm>
								<PstlAdr>
									<AdrLine>Kyriba</AdrLine>
									<AdrLine>L&apos;ANCIENNE-ROUTE</AdrLine>
								</PstlAdr>
							</Dbtr>
						</RltdPties>
						<RmtInf>
							<Ustrd>CASH COLLATERAL</Ustrd>
						</RmtInf>
						<RltdDts>
							<IntrBkSttlmDt>2022-10-17</IntrBkSttlmDt>
						</RltdDts>
					</TxDtls>
				</NtryDtls>
				<AddtlNtryInf>VIRT CPTE</AddtlNtryInf>
			</Ntry>
		</Stmt>
	</BkToCstmrStmt>
</Document>
'''


In [ ]:
#@title Generate CAMT53 based on entered forecast data
import re
import xml.etree.ElementTree as ET

root = ET.fromstring(re.sub('<Document[^>]+', '<Document', camt53, count=1))

get_request = '/v1/accounts/code=' + account
get_response = sample_requests.get_results(token, get_request)
statement_id = js.loads(get_response)['bankAccountID']['statementIdentifier']

if statement_id is None:
	print('\033[91;3mStatement identifier is missing for the account ' + account + '.')
	print('Please add statement identifier and try again.\x1b[0m\n')

for elem in root.iter('Stmt'):
	elem.find('Acct/Id/IBAN').text = statement_id
	
for elem in root.iter('Ntry'):
	elem.find('Amt').text = str(amount)
	elem.find('Amt').set('Ccy', currency) 
	elem.find('BookgDt/Dt').text = transactionDate
	elem.find('ValDt/Dt').text = valueDate
	elem.find('NtryDtls/TxDtls/Refs/EndToEndId').text = reference
	elem.find('NtryDtls/TxDtls/Amt').text = str(amount)
	elem.find('NtryDtls/TxDtls/Amt').set('Ccy', currency)

camt53 = ET.tostring(root)
print(camt53)

                            

In [ ]:
#@title Import CAMT53

now = datetime.now().strftime("%d%m%Y_%H%M%S")
res = sample_requests.import_data(token, camt53, 'API_CAMT53_' + now + '.xml', 'IMP_BK_CAMT53', True, True)


## <font color='yellow'>Monitor cash flow status</font>

In [ ]:
#@title Get status of CAMT53 integration and the cash flow
from google.colab import output
import sys


def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timer = 'The next status check is in {:02d}:{:02d}'.format(mins, secs)
        print('\r' + timer, end='')
        time.sleep(1)
        t -= 1


integ_status = {'Not pushed' : '\033[95mNot pushed \x1b[0m', 
            'Pushed' : '\033[93mPushed \x1b[0m',
            'Integrated' : '\033[92mIntegrated \x1b[0m', 
            'Rejected' : '\033[91mRejected \x1b[0m'}

next_act = {'Not pushed' : 'Pushed', 
            'Pushed' : 'Integrated or Rejected',
            'Integrated' : 'None', 
            'Rejected' : 'None'}

get_req = '/v1/cash-flows/' + res2['uuid'] + '/detail'

#Checking cash flow status every 30 seconds unless it is "ACTUAL"
try:
  while status != 'ACTUAL':

    #Monitoring of bank statement integration
    print("Status of the report:")
    res = sample_requests.run_report(token,"BK_CONNECT_API")
    buffer = sample_requests.StringIO(res[3:])
    df = sample_requests.pd.read_csv(buffer)
    rowIndex = df.index[df['Account'] == account].tolist()[0]

    print('Bank statement delivery status: ' + integ_status[df.at[rowIndex,'Statement delivery']])
    print('Bank statement collection date and time: ' + str(df.at[rowIndex,'Connection time']))
    print('Next bank statement delivery status: ' + next_act[df.at[rowIndex,'Statement delivery']])
      
    #Monitoring of cash flow status
    get_response = sample_requests.get_results(token, get_req)
    json_object= js.loads(get_response)
    status = json_object["status"]
    status_colored = '\033[92mACTUAL\x1b[0m' if status == 'ACTUAL' else '\033[93m' + status + '\x1b[0m'
    print('\nCash flow status: ' + status_colored + '\n')
    if status != 'ACTUAL':
      print('---------------------\n')
      countdown(int(30))
      output.clear()

  print('Forecast with reference ' + reference + ' has been reconciled.')

except IndexError:
    print('\033[91;3mAccount code has not been found in the report.\nPlease check that the statement identifier is in place.\x1b[0m')
